## Implementation of 
# NEURAL NETWORK HYPERSPECTRAL UNMIXING WITH SPECTRAL INFORMATION DIVERGENCE OBJECTIVE*

F. Palsson, J. Sigurdsson, J. R. Sveinsson and M. O. Ulfarsson, "Neural network hyperspectral unmixing with spectral information divergence objective," 2017 IEEE International Geoscience and Remote Sensing Symposium (IGARSS), 2017, pp. 755-758, doi: 10.1109/IGARSS.2017.8127062.

### Imports

In [ ]:
import tensorflow as tf
from tensorflow.keras import initializers, constraints, layers, activations, regularizers
from tensorflow.python.ops import math_ops
from tensorflow.python.keras import backend as K
from tensorflow.python.framework import tensor_shape
from unmixing import HSI, plotEndmembers,SAD
from unmixing import plotEndmembersAndGT, plotAbundancesSimple, load_HSI, PlotWhileTraining
from scipy import io as sio
import os
import numpy as np
from numpy.linalg import inv
import warnings
import matplotlib

warnings.filterwarnings("ignore")
%matplotlib inline

### Use CPU

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

## Class SparseReLU
Performs dynamic thresholding of abundances.

In [ ]:
class SparseReLU(tf.keras.layers.Layer):
    def __init__(self,params):
        self.params=params
        super(SparseReLU, self).__init__()
        self.alpha = self.add_weight(shape=(self.params['num_endmembers'],),initializer=tf.keras.initializers.Zeros(),
        trainable=True, constraint=tf.keras.constraints.non_neg())
    def build(self, input_shape):
        self.alpha = self.add_weight(shape=input_shape[1:],initializer=tf.keras.initializers.Zeros(),
        trainable=True, constraint=tf.keras.constraints.non_neg())
        super(SparseReLU, self).build(input_shape)
    def call(self, x):
        return tf.keras.backend.relu(x - self.alpha)


## Class SumToOne
Performs abundance normalization to enforce ASC

In [ ]:
class SumToOne(layers.Layer):
    def __init__(self, **kwargs):
        super(SumToOne, self).__init__(**kwargs)
        
    def call(self, x):
        x *= K.cast(x >= K.epsilon(), K.floatx())
        x = K.relu(x)
        x = x/(K.sum(x, axis=-1, keepdims=True)+K.epsilon())
        return x

## SID Loss function implementation

In [ ]:
def SID(y_true, y_pred):
    y_true = K.switch(K.min(y_true) < 0, y_true - K.min(y_true) + K.epsilon(), y_true + K.epsilon())
    y_pred = K.switch(K.min(y_pred) < 0, y_pred - K.min(y_pred) + K.epsilon(), y_pred + K.epsilon())

    p_n = y_true / K.sum(y_true, axis=1, keepdims=True)
    q_n = y_pred / K.sum(y_pred, axis=1, keepdims=True)
    return K.sum(p_n * K.log(p_n / q_n)) + K.sum(q_n * K.log(q_n / p_n))


## Class Autoencoder
Wrapper class for the autoencoder model and associcated utility functions

In [ ]:
class Autoencoder(object):
    def __init__(self, params):
        self.data = params["data"].array()
        self.params = params
        self.decoder = layers.Dense(
            units=self.params["n_bands"],
            kernel_regularizer=None,
            activation="linear",
            name="output",
            use_bias=False,
            kernel_constraint=constraints.non_neg()
        )
        self.hidden = layers.Dense(
            units=self.params["num_endmembers"],
            activation='linear',
            name='hidden1',
            use_bias=False
        )
        self.sparseReLU = SparseReLU(params)
        self.asc_layer = SumToOne(name='abundances')
        self.model = self.create_model()
        self.model.compile(optimizer=self.params["optimizer"], loss=self.params["loss"])
        
    def create_model(self):
        input_features = layers.Input(shape=(self.params["n_bands"],))
        code = self.hidden(input_features)
        code = layers.BatchNormalization()(code)
        code = self.sparseReLU(code)
        abunds = self.asc_layer(code)
        output = self.decoder(abunds)

        return tf.keras.Model(inputs=input_features, outputs=output)
        
    
    def fit(self,data,n):
        plot_callback = PlotWhileTraining(n,self.params['data'])
        return self.model.fit(
            x=data,
            y=data,
            batch_size=self.params["batch_size"],
            epochs=self.params["epochs"],
            callbacks=[plot_callback]
        )

    def get_endmembers(self):
        return self.model.layers[len(self.model.layers) - 1].get_weights()[0]

    def get_abundances(self):
        intermediate_layer_model = tf.keras.Model(
            inputs=self.model.input, outputs=self.model.get_layer("abundances").output
        )
        abundances = intermediate_layer_model.predict(self.data)
        abundances = np.reshape(abundances,[self.params['data'].cols,self.params['data'].rows,self.params['num_endmembers']])
        
        return abundances

## Set Hyperparameters and load data

In [ ]:
#Dictonary of aliases for datasets. The first string is the key and second is value (name of matfile without .mat suffix)
#Useful when looping over datasets
datasetnames = {"Urban": "Urban4"
}

dataset = "Urban"

hsi = load_HSI(
    "./Datasets/" + datasetnames[dataset] + ".mat"
)

# Hyperparameters
num_endmembers = 4
num_spectra = 2000
batch_size = 5
learning_rate = 0.001
epochs = 40
loss = SID
opt = tf.optimizers.RMSprop(learning_rate=learning_rate)

data = hsi.array()

# Hyperparameter dictionary
params = {
    "num_endmembers": num_endmembers,
    "batch_size": batch_size,
    "num_spectra": num_spectra,
    "data": hsi,
    "epochs": epochs,
    "n_bands": hsi.bands,
    "GT": hsi.gt,
    "lr": learning_rate,
    "optimizer": opt,
    "loss": loss,
}

plot_every = 0 #Plot endmembers and abundance maps every x epochs. Set to 0 when running experiments. 

training_data = data[
    np.random.randint(0, data.shape[0], num_spectra), :
]


## Train Autoencoder

In [ ]:
autoencoder = Autoencoder(params)
autoencoder.fit(training_data,plot_every)
endmembers = autoencoder.get_endmembers()
abundances = autoencoder.get_abundances()
plotEndmembersAndGT(endmembers, hsi.gt)
plotAbundancesSimple(abundances,'abund.png')


## Run experiment 

In [ ]:
num_runs = 25
plot_every = 0 #Plot endmembers and abundance maps every x epochs. Set to 0 when running experiments. 

results_folder = './Results'

method_name = 'SIDAEU'

for dataset in ['Urban']:
    save_folder = results_folder+'/'+method_name+'/'+dataset
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)

    hsi = load_HSI(
        "./Datasets/" + datasetnames[dataset] + ".mat"
    )
    data=hsi.array()
    batch_size = 256
    params['data']=hsi
    params['n_bands']=hsi.bands

    for run in range(1,num_runs+1):
        training_data = data[np.random.randint(0, data.shape[0], num_spectra), :]
        save_name = dataset+'_run'+str(run)+'.mat'
        save_path = save_folder+'/'+save_name
        autoencoder = Autoencoder(params)
        autoencoder.fit(training_data,plot_every)
        endmembers = autoencoder.get_endmembers()
        abundances = autoencoder.get_abundances()
        plotEndmembersAndGT(endmembers, hsi.gt)
        plotAbundancesSimple(abundances,'abund.png')
        sio.savemat(save_path,{'M':endmembers,'A':abundances})
        del autoencoder